# Pacific Decadal Oscillation - PDO index

Mantua et al. (1997)

> The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)

1. compute monthly global mean SST, deseasonalize it

2. North Pacific monthly output fields

    2.1. create monthly field, determine extend of grid, limit all coordinates appropriately, save as single file
        
        a) North of 0 deg N
        
        b) North of 20 deg N
    
    2.2. deseasonalize and remove global mean, deseasonalized SST, remove mean

3. EOF analysis

maybe use ocn rect for it

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, global_ocean, gl_ocean_rect, mask_box_in_region
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev, notch
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D

### 1. global mean SST temperature series

In [ ]:
# %%time
# # 29 min
# TAREA = xr_AREA('ocn')
# MASK_ocn = boolean_mask('ocn', 0)
# global_area = TAREA.where(MASK_ocn).sum()
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=TAREA, index_loc=global_ocean, AREA_index=global_area, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_{run}.nc')

In [ ]:
# %%time
# # # 7 min for both
# AREA_rect = xr_AREA('ocn_rect')
# MASK_rect = boolean_mask('ocn_rect', 0)
# global_area2 = AREA_rect.where(MASK_rect).sum()
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn_rect', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=AREA_rect, index_loc=gl_ocean_rect, AREA_index=global_area2, MASK=MASK_rect, dims=('t_lat', 't_lon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_rect_{run}.nc')

In [ ]:
# SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
# SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_rect_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_gm_rect_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
# deseasonalize
SST_gm_rect_ds_ctrl = lowpass(lowpass(notch(SST_gm_rect_ctrl, 12), 12),12)
SST_gm_rect_ds_rcp  = lowpass(lowpass(notch(SST_gm_rect_rcp , 12), 12),12)

In [ ]:
# lowpass deseasonalizing
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
# plt.axhline(0, c='k', lw=.5)

plt.plot(SST_gm_rect_ctrl.time[:120]/12   , SST_gm_rect_ctrl   [:120])
plt.plot(SST_gm_rect_ctrl.time[:120]/12   , SST_gm_rect_ds_ctrl[:120])
plt.plot(SST_gm_rect_rcp .time[:120]/12+11, SST_gm_rect_rcp    [:120])
plt.plot(SST_gm_rect_rcp .time[:120]/12+11, SST_gm_rect_ds_rcp [:120])
plt.tight_layout()

In [ ]:
# lowpass deseasonalizing
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
# plt.axhline(0, c='k', lw=.5)

plt.plot(SST_gm_rect_ctrl.time/12    , SST_gm_rect_ctrl   )
plt.plot(SST_gm_rect_ctrl.time/12    , SST_gm_rect_ds_ctrl)
plt.plot(SST_gm_rect_rcp .time/12+220, SST_gm_rect_rcp    )
plt.plot(SST_gm_rect_rcp .time/12+220, SST_gm_rect_ds_rcp )
plt.tight_layout()

### 2.1. creating file with monthly SST fields in North Pacific

In [ ]:
f, ax = plt.subplots(1,3, figsize=(12,4), sharey=True)
for i, r in enumerate(['Pac_40S', 'Pac_Eq', 'Pac_20N']):
    latS = [-38,0,20][i]
    Pac_MASK = mask_box_in_region(domain='ocn_rect', mask_nr=2, bounding_lats=(latS,70), bounding_lons=(110,285))
    Pac_MASK.plot(ax=ax[i])
plt.tight_layout()

In [ ]:
# %%time
# # 8 mins for 200 years of ctrl
# n=1

# for j, run in enumerate(['ctrl', 'rcp']):
#     for y,m,s in IterateOutputCESM(domain='ocn_rect', tavg='monthly', run=run):
#         xa = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         if m==1:
#             print(y)
#             xa_out = xa.copy()    
#         else:
#             xa_out = xr.concat([xa_out, xa], dim='time')

#         if m==12:
#             xa_out.to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_{run}_{y}.nc')
# run='ctrl'
# combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_monthly_rect_{run}_*.nc', concat_dim='time', decode_times=False)
# combined.to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_{run}.nc')
# combined.close()
# remove yearly files

In [ ]:
# %%time
# # 6 mins for 200 years of ctrl
# n=1
# for i, r in enumerate(['Pac_38S', 'Pac_Eq', 'Pac_20N']):
#     latS = [-38,0,20][i]
#     Pac_MASK = mask_box_in_region(domain='ocn_rect', mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,285))
#     NPac_area = xr_AREA('ocn_rect').where(Pac_MASK, drop=True)
#     for j, run in enumerate(['ctrl', 'rcp']):
#         for y,m,s in IterateOutputCESM(domain='ocn_rect', tavg='monthly', run=run):
#             xa = xr.open_dataset(s, decode_times=False).TEMP[0,:,:].where(Pac_MASK, drop=True)
#             if m==1:
#                 print(y)
#                 xa_out = xa.copy()    
#             else:
#                 xa_out = xr.concat([xa_out, xa], dim='time')

#             if m==12:
#                 xa_out.to_netcdf(f'{path_samoc}/SST/SST_monthly_{r}_rect_{run}_{y}.nc')

#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_monthly_{r}_rect_{run}_*.nc', concat_dim='time', decode_times=False)
#     combined.to_netcdf(f'{path_samoc}/SST/SST_monthly_{r}_rect_{run}.nc')
#     combined.close()
# # remove yearly files

In [ ]:
SST_Pac_38S_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_38S_rect_ctrl.nc', decode_times=False)
SST_Pac_38S_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_38S_rect_rcp.nc' , decode_times=False)

SST_Pac_Eq_ctrl  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_Eq_rect_ctrl.nc' , decode_times=False)
SST_Pac_Eq_rcp   = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_Eq_rect_rcp.nc'  , decode_times=False)

SST_Pac_20N_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_20N_rect_ctrl.nc', decode_times=False)
SST_Pac_20N_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_20N_rect_rcp.nc' , decode_times=False)

### 2.2.1. deseasonalize monthly SST field

In [ ]:
%%time
# 37 sec
SST_Pac_38S_ds_ctrl = lowpass(lowpass(notch(SST_Pac_38S_ctrl, 12), 12), 12)
SST_Pac_38S_ds_rcp  = lowpass(lowpass(notch(SST_Pac_38S_rcp , 12), 12), 12)

SST_Pac_Eq_ds_ctrl  = lowpass(lowpass(notch(SST_Pac_Eq_ctrl , 12), 12), 12)
SST_Pac_Eq_ds_rcp   = lowpass(lowpass(notch(SST_Pac_Eq_rcp  , 12), 12), 12)

SST_Pac_20N_ds_ctrl = lowpass(lowpass(notch(SST_Pac_20N_ctrl, 12), 12), 12)
SST_Pac_20N_ds_rcp  = lowpass(lowpass(notch(SST_Pac_20N_rcp , 12), 12), 12)

In [ ]:
SST_Pac_38S_ds_ctrl[100,:,:].plot()

In [ ]:
SST_Pac_Eq_ctrl[:120,100,100].plot()
SST_Pac_Eq_ds_ctrl[:120,100,100].plot()

### 2.2.2. detrend with global mean, des. SST

In [ ]:
len(SST_Pac_38S_ds_rcp)
# len(SST_gm_rect_rcp)

In [ ]:
# in the gm timeseries are some months of the last incomplete year 
SST_Pac_38S_ds_dt_ctrl = SST_Pac_38S_ds_ctrl - SST_gm_rect_ds_ctrl[:-7]
SST_Pac_38S_ds_dt_rcp  = SST_Pac_38S_ds_rcp  - SST_gm_rect_ds_rcp[:-1]

SST_Pac_Eq_ds_dt_ctrl  = SST_Pac_Eq_ds_ctrl  - SST_gm_rect_ds_ctrl[:-7]
SST_Pac_Eq_ds_dt_rcp   = SST_Pac_Eq_ds_rcp   - SST_gm_rect_ds_rcp[:-1]

SST_Pac_20N_ds_dt_ctrl = SST_Pac_20N_ds_ctrl - SST_gm_rect_ds_ctrl[:-7]
SST_Pac_20N_ds_dt_rcp  = SST_Pac_20N_ds_rcp  - SST_gm_rect_ds_rcp[:-1]

### 2.2.3. remove remaining mean

In [ ]:
SST_Pac_20N_ds_dt_ctrl[0,:,:].plot()

In [ ]:
SST_Pac_38S_ds_dt_dm_ctrl = SST_Pac_38S_ds_dt_ctrl - SST_Pac_38S_ds_dt_ctrl.mean('time')
SST_Pac_38S_ds_dt_dm_rcp  = SST_Pac_38S_ds_dt_rcp  - SST_Pac_38S_ds_dt_rcp .mean('time')
SST_Pac_Eq_ds_dt_dm_ctrl  = SST_Pac_Eq_ds_dt_ctrl  - SST_Pac_Eq_ds_dt_ctrl .mean('time')
SST_Pac_Eq_ds_dt_dm_rcp   = SST_Pac_Eq_ds_dt_rcp   - SST_Pac_Eq_ds_dt_rcp  .mean('time')
SST_Pac_20N_ds_dt_dm_ctrl = SST_Pac_20N_ds_dt_ctrl - SST_Pac_20N_ds_dt_ctrl.mean('time')
SST_Pac_20N_ds_dt_dm_rcp  = SST_Pac_20N_ds_dt_rcp  - SST_Pac_20N_ds_dt_rcp .mean('time')

In [ ]:
SST_Pac_20N_ds_dt_dm_ctrl[0,:,:].plot()

### 3. EOF analysis

In [ ]:
%%time
# 12 mins for all
# cut off 2 year on either end due to problems with the erroneous trend introduced by the filtering
for j, run in enumerate(['ctrl', 'rcp']):
    object_list = [[SST_Pac_38S_ds_dt_dm_ctrl, SST_Pac_Eq_ds_dt_dm_ctrl, SST_Pac_20N_ds_dt_dm_ctrl],
                   [SST_Pac_38S_ds_dt_dm_rcp , SST_Pac_Eq_ds_dt_dm_rcp , SST_Pac_20N_ds_dt_dm_rcp ]
                  ][j]
    for i, r in enumerate(['Pac_38S', 'Pac_Eq', 'Pac_20N']):
        print(i)
        SST_object = object_list[i]
        fn = f'{path_results}/SST/SST_PDO_EOF_{r}_{run}.nc'
        latS = [-38,0,20][i]
        Pac_MASK = mask_box_in_region(domain='ocn_rect', mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,285))
        Pac_area = xr_AREA('ocn_rect').where(Pac_MASK, drop=True)
        eof_ctrl, pc_ctrl = EOF_SST_analysis(xa=SST_object[24:-24], weights=Pac_area, fn=fn)
# fn = f'{path_results}/SST/NPac_ctrl.nc'
# eof_ctrl, pc_ctrl = EOF_SST_analysis(xa=SST_NPac_rect_ds_dt_ctrl[24:-24], weights=NPac_area, fn=fn)
# fn = f'{path_results}/SST/NPac_rcp.nc'
# eof_rcp , pc_rcp  = EOF_SST_analysis(xa=SST_NPac_rect_ds_dt_rcp [24:-24], weights=NPac_area, fn=fn)

In [ ]:
Pac_38S_ctrl = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_38S_ctrl.nc', decode_times=False)
Pac_38S_rcp  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_38S_rcp.nc' , decode_times=False)

Pac_Eq_ctrl  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_Eq_ctrl.nc' , decode_times=False)
Pac_Eq_rcp   = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_Eq_rcp.nc'  , decode_times=False)

Pac_20N_ctrl = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_20N_ctrl.nc', decode_times=False)
Pac_20N_rcp  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_20N_rcp.nc' , decode_times=False)

In [ ]:
TPI_ctrl = xr.open_dataarray(f'{path_results}/SST/TPI_ctrl.nc', decode_times=False)
TPI_rcp  = xr.open_dataarray(f'{path_results}/SST/TPI_rcp.nc' , decode_times=False)

In [ ]:
TPI_ctrl.plot()

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
L1, = plt.plot(Pac_38S_ctrl.time/12+100, chebychev(Pac_38S_ctrl.pcs, 13*12), c='C0', lw=1, ls='--', label='PC 38S')
L2, = plt.plot(Pac_38S_rcp .time/12+300, chebychev(Pac_38S_rcp .pcs, 13*12), c='C1', lw=1, ls='--' )
L3, = plt.plot(Pac_Eq_ctrl .time/12+100, chebychev(Pac_Eq_ctrl .pcs, 13*12), c='C0', lw=1, ls=':' , label='PC Eq.')
L4, = plt.plot(Pac_Eq_rcp  .time/12+300, chebychev(Pac_Eq_rcp  .pcs, 13*12), c='C1', lw=1, ls=':'  )
L5, = plt.plot(Pac_20N_ctrl.time/12+100, chebychev(Pac_20N_ctrl.pcs, 13*12), c='C0', lw=3, ls='-' , label='PC 20N')
L6, = plt.plot(Pac_20N_rcp .time/12+300, -chebychev(Pac_20N_rcp .pcs, 13*12), c='C1', lw=3, ls='-'  )

L7, = plt.plot(TPI_ctrl.time/365     , 5*chebychev(TPI_ctrl, 13), c='C0', lw=2, ls='-.', label='TPI')
L8, = plt.plot(TPI_rcp .time/365-1700, 5*chebychev(TPI_rcp , 13), c='C1', lw=2, ls='-.')

plt.xlabel('time [years]'             , fontsize=16)
plt.ylabel('PDO/IPO/TPI indices', fontsize=16)
plt.legend(handles=[L1, L3, L5, L7],ncol=4, fontsize=16)
plt.tight_layout()

In [ ]:
SST_rect_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_ctrl.nc', decode_times=False)
SST_rect_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
SST_rect_ds_dt_ctrl = lowpass(lowpass(notch(SST_rect_ctrl, 12), 12), 12) - SST_gm_rect_ds_ctrl[:-7]
SST_rect_ds_dt_rcp  = lowpass(lowpass(notch(SST_rect_rcp , 12), 12), 12) - SST_gm_rect_ds_rcp[:-1]

In [ ]:
SST_rect_ds_dt_ctrl.to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_ctrl.nc')
SST_rect_ds_dt_rcp .to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_rcp.nc' )

## correlation plots

In [ ]:
xa = Pac_20N_ctrl.eofs[0]
cmap = discrete_cmap(14,  shifted_color_map(cmocean.cm.balance, start=0, midpoint=0.5, stop=1., name='shrunk'))
f = plt.figure(figsize=(8,5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
# ax.set_extent([.02,.98,.12,.86])
im1 = ax.pcolormesh(xa.t_lon, xa.t_lat, xa.values,
                    cmap=cmap, vmin=-1, vmax=1,
                    transform=ccrs.PlateCarree() )
ax.add_feature(cartopy.feature.LAND, facecolor='grey', edgecolor='k', alpha=0.5)
cb = plt.colorbar(im1, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 expressed as correlation', fontsize=16)
# plt.savefig(f'{path_results}/SST/PDO_EOF', bbox_inches='tight')
# plt.tight_layout()
# (NPac_rcp.eofs-NPac_ctrl.eofs).plot()

In [ ]:
from xr_regression import lag_linregress_3D

In [ ]:
ds_ctrl = lag_linregress_3D(Pac_20N_ctrl.pcs[:-7,0], SST_rect_ds_dt_ctrl[24:-(24+7)])

In [ ]:
len(Pac_20N_ctrl.pcs[:-7,0])

In [ ]:
len(SST_rect_ds_dt_ctrl)-24-24-7

In [ ]:
Pac_20N_ctrl.pcs[:,0].plot()